# Performances of 2D integration vs 1D integration

This is dependant on:
* Number of azimuthal bins
* Pixel splitting
* Algorithm
* Implementation (i.e. programming language)
* Hardware used

Thus there is no general answer. But here is a quick benchmark to evaluate the penality on performances:

import sys
import os
import time
import numpy
import fabio
import pyFAI
from pyFAI.test.utilstest import UtilsTest
import pyFAI.method_registry
import pyFAI.integrator.azimuthal
print(f"Python version: {sys.version}")
print(f"PyFAI version: {pyFAI.version}")
start_time = time.perf_counter()

In [1]:
import sys
import os
import time
import numpy

os.environ["PYOPENCL_COMPILER_OUTPUT"] = "0"
start_time = time.perf_counter()

In [2]:
import fabio
import pyFAI
from pyFAI.test.utilstest import UtilsTest
import pyFAI.method_registry
import pyFAI.integrator.azimuthal
print(f"Python version: {sys.version}")
print(f"PyFAI version: {pyFAI.version}")


Python version: 3.13.1 | packaged by conda-forge | (main, Jan 13 2025, 09:53:10) [GCC 13.3.0]
PyFAI version: 2025.12.0


In [3]:
print("Number of way to performing integration:", len(pyFAI.method_registry.IntegrationMethod.list_available()))

Number of way to performing integration: 95


In [4]:
ai = pyFAI.load(UtilsTest.getimage("Pilatus1M.poni"))
img = fabio.open(UtilsTest.getimage("Pilatus1M.edf")).data
ai

Detector Pilatus 1M	 PixelSize= 172µm, 172µm	 BottomRight (3)
Wavelength= 1.000000 Å
SampleDetDist= 1.583231e+00 m	PONI= 3.341702e-02, 4.122778e-02 m	rot1=0.006487  rot2=0.007558  rot3=0.000000 rad
DirectBeamDist= 1583.310 mm	Center: x=179.981, y=263.859 pix	Tilt= 0.571° tiltPlanRotation= 130.640° λ= 1.000Å

In [5]:
%%time
#Tune those parameters to match your needs:
kw1 = {"data": img, "npt":1000}
kw2 = {"data": img, "npt_rad":1000}
#Actual benchmark:
res = {}
for k,v in pyFAI.method_registry.IntegrationMethod._registry.items():
    print(k)
    if k.dim == 1:
        res[k] = %timeit -o ai.integrate1d(method=v, **kw1)
    else:
        res[k] = %timeit -o ai.integrate2d(method=v, **kw2)

Method(dim=1, split='no', algo='histogram', impl='python', target=None)
30.1 ms ± 436 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Method(dim=2, split='no', algo='histogram', impl='python', target=None)
115 ms ± 734 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Method(dim=1, split='no', algo='histogram', impl='cython', target=None)
11.2 ms ± 29.2 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Method(dim=2, split='no', algo='histogram', impl='cython', target=None)
16.7 ms ± 1.16 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
Method(dim=1, split='bbox', algo='histogram', impl='cython', target=None)
26.1 ms ± 94.7 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Method(dim=2, split='bbox', algo='histogram', impl='cython', target=None)
32.5 ms ± 48.4 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Method(dim=1, split='full', algo='histogram', impl='cython', target=None)
169 ms ± 3.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops e

1 error generated.
1 error generated.


14.9 ms ± 523 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)
Method(dim=2, split='no', algo='histogram', impl='opencl', target=(1, 0))


/users/kieffer/.venv/py313/lib/python3.13/site-packages/pyopencl/cache.py:496: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  _create_built_program_from_source_cached(
/users/kieffer/.venv/py313/lib/python3.13/site-packages/pyopencl/cache.py:500: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  prg.build(options_bytes, devices)


11.2 ms ± 960 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)
Method(dim=1, split='no', algo='histogram', impl='opencl', target=(2, 0))


11.5 ms ± 254 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Method(dim=2, split='no', algo='histogram', impl='opencl', target=(2, 0))
7.7 ms ± 220 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Method(dim=1, split='bbox', algo='csr', impl='opencl', target=(0, 0))
704 μs ± 2.68 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Method(dim=2, split='bbox', algo='csr', impl='opencl', target=(0, 0))
2.68 ms ± 69.8 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)
Method(dim=1, split='no', algo='csr', impl='opencl', target=(0, 0))
660 μs ± 1.57 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Method(dim=2, split='no', algo='csr', impl='opencl', target=(0, 0))
2.59 ms ± 14.1 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Method(dim=1, split='bbox', algo='csr', impl='opencl', target=(0, 1))
1.22 ms ± 3.48 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Method(dim=2, split='bbox', algo='csr', impl='opencl', target=(0, 1))
6.12 ms 

In [6]:
print("-"*80)
print(f"{'Split':5s} | {'Algo':9s} | {'Impl':6s}| {'1d (ms)':8s} | {'2d (ms)':8s} | {'ratio':6s} | Device")
print("-"*80)
for k in res:
    if k.dim == 1:
        k1 = k
        k2 = k._replace(dim=2)
        if k2 in res:
            print(f"{k1.split:5s} | {k1.algo:9s} | {k1.impl:6s}| {res[k1].best*1000:8.3f} | {res[k2].best*1000:8.3f} | {res[k2].best/res[k1].best:6.1f} | ",
                    end="")
        if k.target:
            print(pyFAI.method_registry.IntegrationMethod._registry.get(k).target_name)
        else:
            print()
print("-"*80)

--------------------------------------------------------------------------------
Split | Algo      | Impl  | 1d (ms)  | 2d (ms)  | ratio  | Device
--------------------------------------------------------------------------------
no    | histogram | python|   29.819 |  113.944 |    3.8 | 
no    | histogram | cython|   11.131 |   16.140 |    1.5 | 
bbox  | histogram | cython|   25.982 |   32.473 |    1.2 | 
full  | histogram | cython|  166.489 |  313.909 |    1.9 | 
no    | csr       | cython|    7.103 |    7.166 |    1.0 | 
bbox  | csr       | cython|    7.468 |    8.253 |    1.1 | 
no    | csr       | python|    9.785 |   14.210 |    1.5 | 
bbox  | csr       | python|   13.559 |   16.487 |    1.2 | 
no    | csc       | cython|    8.101 |   10.589 |    1.3 | 
bbox  | csc       | cython|   10.463 |   13.837 |    1.3 | 
no    | csc       | python|   10.874 |   14.124 |    1.3 | 
bbox  | csc       | python|   14.671 |   21.500 |    1.5 | 
bbox  | lut       | cython|    7.437 |   11.805 |   

In [7]:
print(f"Total runtime: {time.perf_counter()-start_time:.3f}s")

Total runtime: 462.969s
